In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("LINEAR_API_KEY")
API_KEY

In [ ]:
# Cell 1: Setup and Configuration
import requests
import json
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# --- CONFIGURATION ---
API_KEY = os.getenv("LINEAR_API_KEY")

# --- API Setup ---
API_URL = "https://api.linear.app/graphql"
HEADERS = {
    "Authorization": API_KEY, # No "Bearer" prefix
    "Content-Type": "application/json"
}

# Helper function to run GraphQL queries/mutations
def run_linear_query(query, variables=None):
    """A simple function to post a query to the Linear API."""
    payload = {'query': query}
    if variables:
        payload['variables'] = variables
    
    response = requests.post(API_URL, headers=HEADERS, json=payload)
    
    if response.status_code == 200:
        json_response = response.json()
        if "errors" in json_response:
            print(f"❌ GraphQL Error: {json_response['errors']}")
            return None
        return json_response
    else:
        raise Exception(f"Query failed to run with status code {response.status_code}: {response.text}")

if not API_KEY:
    print("❌ ERROR: LINEAR_API_KEY not found in environment variables.")
else:
    print("✅ Setup complete. API key loaded. Ready to proceed.")
    
    # Cell 1.5 (Debug Cell): Find Your New Team Keys

print("🔎 Fetching all teams in your new workspace...")

debug_query = """
  query {
    teams {
      nodes {
        id
        name
        key  # <-- This is the Team Key you need
      }
    }
  }
"""

if API_KEY:
    debug_data = run_linear_query(debug_query)

    if debug_data:
        print("✅ Found the following teams in this workspace:")
        print("--------------------------------------------------")
        for team in debug_data['data']['teams']['nodes']:
            print(f"  - Name: \"{team['name']}\",  Key: \"{team['key']}\"")
        print("--------------------------------------------------")
        print("\n📝 Copy the 'Key' (e.g., 'TEA1') of the team you want to use and paste it into Cell 6.")
else:
    print("🚫 API Key not loaded. Please fix Cell 1.")

In [ ]:
# Cell 2: Function - Fetch All Team Assets (Left Panel)

def fetch_all_team_assets(team_key: str):
    """
    Fetches all projects and issues for a given team.
    This data will populate the left-side panel of the app.
    """
    print(f"🔎 Fetching all assets for team '{team_key}'...")
    query = """
    query($teamKey: String!) {
      team(id: $teamKey) {
        id
        name
        
        projects {
          nodes {
            id
            name
            state
          }
        }
        
        issues {
          nodes {
            id
            title
            identifier
            state { name }
            priority
          }
        }
      }
    }
    """
    variables = {"teamKey": team_key}
    data = run_linear_query(query, variables)
    
    if data and data['data']['team']:
        print("✅ Successfully fetched assets.")
        return data['data']['team']
    else:
        print(f"❌ Could not find team with key '{team_key}'.")
        return None


In [ ]:
# Cell 3: Function - Fetch Workspace Properties (Corrected)

def fetch_workspace_properties(team_key: str):
    """
    Fetches all properties needed to populate the dropdowns
    in the app's right-side editing panel.
    """
    print(f"🔎 Fetching all workspace properties...")
    query = """
    query($teamKey: String!) {
      # 1. Get properties for Issues
      team(id: $teamKey) {
        labels { nodes { id, name } }
        states { nodes { id, name } }
      }
      
      # 2. Get properties for Projects
      organization {
        # --- THIS IS THE CORRECTED PART ---
        # projectStatuses is a list, not a connection, so no "nodes"
        projectStatuses { id, name }
        # ----------------------------------
      }
      
      # 3. Get projects to populate the "Move to Project" dropdown
      projects {
        nodes { id, name }
      }
    }
    """
    variables = {"teamKey": team_key}
    data = run_linear_query(query, variables)
    
    if data:
        print("✅ Successfully fetched properties.")
        # Organize the data
        properties = {
            "issueLabels": data['data']['team']['labels']['nodes'],
            "issueStatuses": data['data']['team']['states']['nodes'],
            # --- THIS IS THE CORRECTED PART ---
            "projectStatuses": data['data']['organization']['projectStatuses'],
            # ----------------------------------
            "allProjects": data['data']['projects']['nodes'],
            "issuePriorities": [
                {"name": "No priority", "value": 0},
                {"name": "Urgent", "value": 1},
                {"name": "High", "value": 2},
                {"name": "Medium", "value": 3},
                {"name": "Low", "value": 4}
            ]
        }
        return properties
    else:
        print("❌ Could not fetch workspace properties.")
        return None

In [ ]:
# Cell 4: Function - Bulk Update Projects

def bulk_update_projects(project_ids: list, updates: dict):
    """
    Updates a list of projects with the provided properties.
    
    'updates' dict can contain:
    - state: "Planned", "In Progress", etc.
    - priority: 0-4
    - startDate: "YYYY-MM-DD"
    - targetDate: "YYYY-MM-DD"
    - labelIds: ["id1", "id2"]
    """
    print(f"\n🚀 Starting bulk update for {len(project_ids)} project(s)...")
    
    mutation = """
    mutation ProjectUpdate($id: String!, $input: ProjectUpdateInput!) {
      projectUpdate(id: $id, input: $input) {
        success
        project { id, name, state }
      }
    }
    """
    
    # The 'updates' dict is our $input variable
    # We must convert date strings to full timestamps
    if "startDate" in updates:
        updates["startDate"] = datetime.fromisoformat(updates["startDate"]).isoformat()
    if "targetDate" in updates:
        updates["targetDate"] = datetime.fromisoformat(updates["targetDate"]).isoformat()
    
    
    for pid in project_ids:
        print(f"  - Updating project {pid}...")
        variables = {"id": pid, "input": updates}
        data = run_linear_query(mutation, variables)
        if data and data['data']['projectUpdate']['success']:
            print(f"    ✅ Success: '{data['data']['projectUpdate']['project']['name']}' updated.")
        else:
            print(f"    ❌ Failed to update {pid}.")
            
    print("✨ Bulk project update complete.")

In [ ]:
# Cell 5: Function - Bulk Update Issues

def bulk_update_issues(issue_ids: list, updates: dict):
    """
    Updates a list of issues with the provided properties.
    
    'updates' dict can contain:
    - stateId: "state-uuid"
    - priority: 0-4
    - projectId: "project-uuid"
    - labelIds: ["id1", "id2"]
    """
    print(f"\n🚀 Starting bulk update for {len(issue_ids)} issue(s)...")
    
    mutation = """
    mutation IssueUpdate($id: String!, $input: IssueUpdateInput!) {
      issueUpdate(id: $id, input: $input) {
        success
        issue { id, identifier, state { name } }
      }
    }
    """
    
    # The 'updates' dict is our $input variable
    
    for iid in issue_ids:
        print(f"  - Updating issue {iid}...")
        variables = {"id": iid, "input": updates}
        data = run_linear_query(mutation, variables)
        if data and data['data']['issueUpdate']['success']:
            print(f"    ✅ Success: '{data['data']['issueUpdate']['issue']['identifier']}' updated.")
        else:
            print(f"    ❌ Failed to update {iid}.")
            
    print("✨ Bulk issue update complete.")

In [ ]:
# Cell 6: Test - Setup (Fetch All Data)

# ⚠️ Set your team key here
TEAM_KEY = "LIN"

if API_KEY:
    # 1. Fetch all projects and issues
    team_assets = fetch_all_team_assets(TEAM_KEY)
    
    # 2. Fetch all available properties
    workspace_properties = fetch_workspace_properties(TEAM_KEY)

    if team_assets and workspace_properties:
        print("\n--- ✅ SETUP COMPLETE ---")
        
        # --- Print some sample data you can use for the test ---
        print("\nSample Projects:")
        for p in team_assets['projects']['nodes'][:2]: # Print first 2
            print(f"  - Name: {p['name']}, ID: {p['id']}")
            
        print("\nSample Issues:")
        for i in team_assets['issues']['nodes'][:2]: # Print first 2
            print(f"  - Name: {i['title']}, ID: {i['id']}")

        print("\nSample Issue Labels:")
        for l in workspace_properties['issueLabels'][:2]:
            print(f"  - Name: {l['name']}, ID: {l['id']}")
            
        print("\nSample Issue Statuses (for 'stateId'):")
        for s in workspace_properties['issueStatuses'][:2]:
            print(f"  - Name: {s['name']}, ID: {s['id']}")
            
        print("\nSample Project Statuses (for 'state'):")
        for s in workspace_properties['projectStatuses']:
            print(f"  - Name: {s['name']}") # No 'id' needed for project state
else:
    print("🚫 API Key not loaded. Please fix Cell 1.")

In [ ]:
# Cell 7: Test - Run Bulk Updates

if 'team_assets' in locals() and 'workspace_properties' in locals():

    # --- 1. TEST BULK PROJECT UPDATE ---

    project_ids_to_update = [
        "283bb14d-cf61-43ee-a582-c23aa9b473a7", # HydraTrack
        "cbb925bc-c298-49ad-bee0-574a63b5e65b"  # Hydrate Tracker
    ]

    project_changes = {
        # --- THIS IS THE CORRECTED LINE ---
        "state": "completed", # Use lowercase 'completed' (the state type)
        # ----------------------------------
        "targetDate": "2025-10-31" # YYYY-MM-DD format
    }

    print("--- STARTING PROJECT UPDATE ---")
    bulk_update_projects(project_ids_to_update, project_changes)


    # --- 2. TEST BULK ISSUE UPDATE ---

    issue_ids_to_update = [
        "2a0c5bc5-6f70-4cef-be23-09bde52de296", # Create Discord Bot 2
        "7c283404-4307-4e2a-81cc-ffeeaceaf265"  # Create Discord Bot
    ]

    test_label_id = "a0eb1457-123e-4e78-ac84-662843844714" # Bug
    test_state_id = "e7ab3f79-df00-4436-a160-1d4494fa47d8" # In Progress

    issue_changes = {
        "stateId": test_state_id,
        "labelIds": [test_label_id] # Must be a list
    }

    print("\n--- STARTING ISSUE UPDATE ---")
    bulk_update_issues(issue_ids_to_update, issue_changes)

else:
    print("🚫 Data not loaded. Please run Cell 6 first.")